In [3]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np

In [4]:
# Let's import Keras
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Dataset Preparation and Pre-Processing

In [ ]:
# Get image dimensions 

# Split into training and testing data 
import os
import cv2
import sklearn

xdata = []
ydata = []

for piece in os.listdir("chess_dataset"):
    for image in os.listdir(os.path.join("chess_dataset", piece)):
        if image.startswith('.'):
            continue
        src = os.path.join("chess_dataset", piece, image)
        img = cv2.imread(src, cv2.IMREAD_GRAYSCALE)
        img_rows, img_cols = img.shape
        ydata.append(piece)
        xdata.append(img)

xdata = np.array(xdata)
ydata = np.array(ydata)

RANDOM_SEED = 7
    
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(
    xdata, ydata, test_size=0.3, random_state=RANDOM_SEED, stratify=ydata
)

print(len(x_train), len(x_test), len(y_train), len(y_test))

# change data shape to be correct for keras
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# change data type to 32-bit float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# scale data values to be between 0 and 1
x_train /= 255
x_test /= 255




print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')




# Convert to one-hot labels 

num_classes = 5                # THIS NUMBER SHOULD CHANGE 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


455 196 455 196


AttributeError: 'list' object has no attribute 'reshape'

Build the model

In [ ]:
# Prepares a model of a Neural Network that we will fit data to
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Train the model using the training data 

In [ ]:
# setup loss function (cross-entropy) and optimiser (adam)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])  # try 'fmeasure' for f1 score
print(model.summary())
# Conv2D layers: have biases by default
# each filter (height x width x input channels) -> scalar response
# accounting for padding too, out: (height (pad) x width (pad) x filters)

# train the model!
batch_size = 128
epochs = 12

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Model evaluation 

In [ ]:
# evaluate trained model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# make predictions on the test data
predictions = model.predict(x_test)
predictions = np.argmax(predictions,axis=1)

labels = np.argmax( y_test , axis=1 )